# DisplacementMIP - Flood displacement impacts in the Horn of Africa

We model the annual average displacment (AAD) impacts and return periods curves for displaced people in the horn of Africa (Ethopia, Somalia, Sudan) for the displacmentMIP.

Data used:
* flood hazard maps from CIMA
* building exposure model (BEM) from [UNEP-GRID](https://giri.unepgrid.ch/sites/default/files/2023-09/GIRI_BEM_report_UNIGE.pdf)

Set country (Ethopia, Somalia, Sudan (SDN)); note, we do not include South Sudan (SSD).

In [36]:
# hazard
CNTRY = 'SOMALIA'

In [37]:
# exposure
cntry_name = 'Somalia'

## BEM exposure model

Load Exposures

In [3]:
from climada.entity.exposures import Exposures

In [4]:
import numpy as np
import os

os.chdir('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk') # change back to root folder, not "~/doc"
import exposure
import vulnerability

### BEM subcomponents - prep for use with IVM impact functions

In [31]:
# Load the full dataframe, without further re-aggregation / processing other than adding centroids
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_name, opt='full')
gdf_bem_subcomps.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry
0,174025676,SOM,3,beds_priv,W2,0.000342,5.179267e-07,100.0,0.0,0.0,POINT (50.54583 11.86250)
1,174025676,SOM,3,beds_priv,W1,0.000146,2.214579e-07,100.0,0.0,0.0,POINT (50.54583 11.86250)
2,174025676,SOM,3,beds_priv,UCB,0.000005,7.143810e-09,100.0,0.0,0.0,POINT (50.54583 11.86250)
3,174025676,SOM,3,beds_priv,RS2,0.000038,5.715046e-08,100.0,0.0,0.0,POINT (50.54583 11.86250)
4,174025676,SOM,3,beds_priv,INF,0.000014,2.143141e-08,100.0,0.0,0.0,POINT (50.54583 11.86250)


In [32]:
# filter and apply impf id
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>0.001] # filter out rows with basically no population
gdf_bem_subcomps['impf_FL'] = gdf_bem_subcomps.apply(lambda row: vulnerability.DICT_PAGER_FLIMPF_IVM[row.se_seismo], axis=1)

In [33]:
# replace impf 3 --> 5 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==3), "impf_FL"] = 5

In [34]:
# replace impf 4 --> 6 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==4), "impf_FL"] = 6

In [35]:
# remove for now unnecessary cols and prepare gdf for CLIMADA Exposure
gdf_bem_subcomps.rename({'valhum' : 'value'}, axis=1)
for col in ['iso3', 'sector', 'valfis', 'se_seismo']:
    gdf_bem_subcomps.pop(col)
gdf_bem_subcomps

,id_1x,cpx,valhum,bd_1_floor,bd_2_floor,bd_3_floor,geometry,impf_FL
6,194457751,3,0.003283,0.000,0.000,0.0,POINT (43.40417 0.66250),2
7,194457751,3,0.001404,0.000,0.000,0.0,POINT (43.40417 0.66250),2
12,194457751,3,0.002711,0.000,0.000,0.0,POINT (43.40417 0.66250),2
13,194457751,3,0.001159,0.000,0.000,0.0,POINT (43.40417 0.66250),2
18,194457751,3,0.281569,0.000,0.000,0.0,POINT (43.40417 0.66250),2
...,...,...,...,...,...,...,...,...
6085408,194650976,3,2.444534,78.022,21.978,0.0,POINT (41.59583 -1.64583),2
6085409,194650976,3,0.078856,78.022,21.978,0.0,POINT (41.59583 -1.64583),6
6085410,194650976,3,0.630848,78.022,21.978,0.0,POINT (41.59583 -1.64583),5
6085411,194650976,3,0.236568,78.022,21.978,0.0,POINT (41.59583 -1.64583),1


Make CLIMADA Exposure with mutliply defined centroids

In [36]:
exp_bem = Exposures(gdf_bem_subcomps)
exp_bem.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp_bem.value_unit = 'Pop. count'
exp_bem.gdf['longitude'] = exp_bem.gdf.geometry.x
exp_bem.gdf['latitude'] = exp_bem.gdf.geometry.y
exp_bem.gdf = exp_bem.gdf[~np.isnan(
    exp_bem.gdf.latitude)]  # drop nan centroids
exp_bem.gdf.head()

,id_1x,cpx,value,bd_1_floor,bd_2_floor,bd_3_floor,geometry,impf_FL,longitude,latitude
6,194457751,3,0.003283,0.0,0.0,0.0,POINT (43.40417 0.66250),2,43.404167,0.6625
7,194457751,3,0.001404,0.0,0.0,0.0,POINT (43.40417 0.66250),2,43.404167,0.6625
12,194457751,3,0.002711,0.0,0.0,0.0,POINT (43.40417 0.66250),2,43.404167,0.6625
13,194457751,3,0.001159,0.0,0.0,0.0,POINT (43.40417 0.66250),2,43.404167,0.6625
18,194457751,3,0.281569,0.0,0.0,0.0,POINT (43.40417 0.66250),2,43.404167,0.6625


In [37]:
print('Total population in exposure layer: ' + "{:,.0f}".format(exp_bem.gdf.value.sum()))

Total population in exposure layer: 19,054,740


The exposure layer for Somalia includes disputed areas and Somaliland. We thus crop the extent of the exposure to the natural earth polygon.

In [38]:
import shapely
import geopandas as gpd
from climada.util import coordinates as u_coords
cntry_iso = u_coords.country_to_iso(cntry_name)
geom_cntry = shapely.ops.unary_union(
    [geom for geom in
     u_coords.get_country_geometries([cntry_iso]).geometry])
polygon_gdf = gpd.GeoDataFrame([1], geometry=[geom_cntry], crs=gdf_bem_subcomps.crs)
overlay_bem = gpd.sjoin(gdf_bem_subcomps, polygon_gdf, how="inner", op="intersects")

print('Total population in cropped exposure layer: ' + "{:,.0f}".format(overlay_bem.value.sum()))

exp = Exposures(overlay_bem)

/Users/simonameiler/opt/anaconda3/envs/climada_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Total population in cropped exposure layer: 14,134,633


In [39]:
print('Total population in exposure layer: ' + "{:,.0f}".format(exp.gdf.value.sum()))

Total population in exposure layer: 14,134,633


In [40]:
exp.write_hdf5(f'{cntry_name}_BEM_exp_IVM.hdf5')

### BEM subcomponents - prep for use with IVM impact functions

In [16]:
# Load the full dataframe, without further re-aggregation / processing other than adding centroids
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_name, opt='full')
gdf_bem_subcomps.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry
0,140485949,SDN,3,beds_priv,W,0.000311,0.000001,85.106,6.384,8.51,None
1,140485949,SDN,3,beds_priv,UFB,0.006213,0.000021,85.106,6.384,8.51,None
2,140485949,SDN,3,beds_priv,M,0.012115,0.000041,85.106,6.384,8.51,None
3,140485949,SDN,3,beds_priv,INF,0.026716,0.000090,85.106,6.384,8.51,None
4,140485949,SDN,3,beds_priv,A,0.008388,0.000028,85.106,6.384,8.51,None


In [17]:
# filter and apply impf id
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>0.001] # filter out rows with basically no population
gdf_bem_subcomps['impf_FL'] = gdf_bem_subcomps.apply(lambda row: vulnerability.DICT_PAGER_FLIMPF_CIMA[row.se_seismo], axis=1)

In [18]:
# remove for now unnecessary cols and prepare gdf for CLIMADA Exposure
gdf_bem_subcomps.rename({'valhum' : 'value'}, axis=1)
for col in ['iso3', 'sector', 'valfis', 'se_seismo', 'bd_1_floor','bd_2_floor','bd_3_floor']:
    gdf_bem_subcomps.pop(col)

Make CLIMADA Exposure with mutliply defined centroids

In [19]:
exp_bem = Exposures(gdf_bem_subcomps)
exp_bem.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp_bem.value_unit = 'Pop. count'
exp_bem.gdf['longitude'] = exp_bem.gdf.geometry.x
exp_bem.gdf['latitude'] = exp_bem.gdf.geometry.y
exp_bem.gdf = exp_bem.gdf[~np.isnan(
    exp_bem.gdf.latitude)]  # drop nan centroids
exp_bem.gdf.head()

,id_1x,cpx,value,geometry,impf_FL,longitude,latitude
16,169203556,3,0.005079,POINT (33.77917 17.33750),6,33.779167,17.3375
17,169203556,3,0.009903,POINT (33.77917 17.33750),12,33.779167,17.3375
18,169203556,3,0.021838,POINT (33.77917 17.33750),14,33.779167,17.3375
19,169203556,3,0.006856,POINT (33.77917 17.33750),12,33.779167,17.3375
20,169203556,3,0.002218,POINT (33.77917 17.33750),14,33.779167,17.3375


In [20]:
print('Total population in exposure layer: ' + "{:,.0f}".format(exp_bem.gdf.value.sum()))

Total population in exposure layer: 59,880,402


The exposure layer for Somalia includes disputed areas and Somaliland. We thus crop the extent of the exposure to the natural earth polygon.

In [21]:
import shapely
import geopandas as gpd
from climada.util import coordinates as u_coords
cntry_iso = u_coords.country_to_iso(cntry_name)
geom_cntry = shapely.ops.unary_union(
    [geom for geom in
     u_coords.get_country_geometries([cntry_iso]).geometry])
polygon_gdf = gpd.GeoDataFrame([1], geometry=[geom_cntry], crs=gdf_bem_subcomps.crs)
overlay_bem = gpd.sjoin(gdf_bem_subcomps, polygon_gdf, how="inner", op="intersects")

print('Total population in cropped exposure layer: ' + "{:,.0f}".format(overlay_bem.value.sum()))

exp = Exposures(overlay_bem)

/Users/simonameiler/opt/anaconda3/envs/climada_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Total population in cropped exposure layer: 59,771,246


In [22]:
print('Total population in exposure layer: ' + "{:,.0f}".format(exp.gdf.value.sum()))

Total population in exposure layer: 59,771,246


In [23]:
exp.write_hdf5(f'{cntry_name}_BEM_exp_CIMA.hdf5')